In [2]:
from IPython.display import display
import sqlite3
import pandas as pd

## **Importar datos**

In [3]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('/home/isa/sumz/workshop_DE/database/db_workshop.db')

## **Productos más populares cada mes durante el año 2018**

In [5]:
# Ejecutar una consulta SQL con pandas y obtener un DataFrame
df_productos_populares_2018 = pd.read_sql_query(
    """WITH sales_per_month AS (
    SELECT 
        strftime('%m', date) AS month,
        product_id,
        SUM(sales) AS total_sales
    FROM sales
    WHERE strftime('%Y', date) = "2018"
    GROUP BY 1, 2
),
ranked_sales AS (
    SELECT 
        month,
        product_id,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY month ORDER BY total_sales DESC) AS rank
    FROM sales_per_month
)
SELECT month, product_id, total_sales
FROM ranked_sales
WHERE rank <= 5
ORDER BY MONTH ASC, total_sales DESC""", conn)

pd.set_option('display.max_rows', 10)
display(df_productos_populares_2018)

,month,product_id,total_sales
0,01,P0438,21326.0
1,01,P0103,19046.0
2,01,P0364,7984.0
3,01,P0051,7754.0
4,01,P0590,5692.0
...,...,...,...
55,12,P0103,19405.0
56,12,P0388,12112.0
57,12,P0364,12000.0
58,12,P0051,9066.0


![Evidencia](https://raw.githubusercontent.com/isantrich/workshop_DE/refs/heads/main/images_dbeaver/productos_populares_2018.png)

## **En cuántas tiendas se implementaron promociones del tipo 1 en el canal 1 durante el año 2019**

In [3]:
df_promo_1_canal_1_2019 = pd.read_sql_query(
    """
    SELECT COUNT(DISTINCT store_id) AS promo_tipo_1_canal_1
    FROM sales
    WHERE promo_type_1 IS NOT NULL
    AND strftime('%Y', date) = "2019";
    """, conn)
display(df_promo_1_canal_1_2019)

,promo_tipo_1_canal_1
0,144


![Evidencia](https://raw.githubusercontent.com/isantrich/workshop_DE/refs/heads/main/images_dbeaver/promo_tipo_1_canal_1.png)

## **Cuál fue la ciudad que tuvo el mayor volumen de ventas en general durante todo el periodo de 2017 a 2019**

In [4]:
ciudad_mayor_volumen_ventas_2017_2019 = pd.read_sql_query(
    """
    SELECT sc.city_id, SUM(s.sales) total_sales
    FROM sales s
    LEFT JOIN store_cities sc 
    ON s.store_id = sc.store_id 
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
    """, conn)
display(ciudad_mayor_volumen_ventas_2017_2019)

,city_id,total_sales
0,C014,2573439.552


![Evidencia](https://raw.githubusercontent.com/isantrich/workshop_DE/refs/heads/main/images_dbeaver/ciudad_mayor_volumen_ventas.png)

## **Cuál fue el promedio de stock disponible en todas las tiendas durante el año 2017**

In [5]:
promedio_stock_tienda = pd.read_sql_query(
    """
    WITH promedio_por_tienda AS(
    SELECT store_id, AVG(stock) promedio_tiendas
    FROM sales 
    WHERE strftime('%Y', date) = "2017"
    GROUP BY 1
    )
    SELECT ROUND(AVG(promedio_tiendas),2) AS promedio_stock_general_tiendas
    FROM promedio_por_tienda
    """, conn)
display(promedio_stock_tienda)

,promedio_stock_general_tiendas
0,15.85


![Evidencia](https://raw.githubusercontent.com/isantrich/workshop_DE/refs/heads/main/images_dbeaver/promedio_stock_tienda.png)

## **Cuántas veces se aplicaron descuentos en el canal 2 durante el mes de diciembre de 2018**

In [6]:
descuentos_canal_2_2018 = pd.read_sql_query(
    """
    SELECT count(promo_type_2) AS descuento_canal_2
    FROM sales 
    WHERE strftime('%Y', date) = "2018"
    AND strftime('%m', date) = "12"
    AND promo_type_2 IS NOT NULL
    AND promo_discount_2 <> ""
    """, conn)
display(descuentos_canal_2_2018)

,descuento_canal_2
0,0


## **Cuál fue el producto con el precio de venta más alto en cada tienda durante el tercer trimestre de 2019**

In [8]:
precio_venta_mas_alto = pd.read_sql_query(
    """
WITH AGRUPADO AS (
SELECT store_id, product_id, MAX(price) AS precio_maximo
FROM sales
WHERE date BETWEEN '2019-07-01' AND '2019-09-30'
AND price <> ""
GROUP BY 1, 2
),
RANKING AS (
SELECT store_id, product_id, precio_maximo,
ROW_NUMBER() OVER ( PARTITION BY store_id ORDER BY precio_maximo DESC) AS fila
FROM AGRUPADO
)
SELECT store_id, product_id, precio_maximo
FROM RANKING
WHERE  fila= 1
ORDER BY 1, 3 DESC

    """, conn)
display(precio_venta_mas_alto)

,store_id,product_id,precio_maximo
0,S0001,P0498,299.90
1,S0002,P0708,1599.00
2,S0003,P0498,299.90
3,S0004,P0498,299.90
4,S0005,P0498,299.90
...,...,...,...
139,S0140,P0680,139.95
140,S0141,P0498,299.90
141,S0142,P0392,199.90
142,S0143,P0498,299.90


## **Cuántas tiendas experimentaron un aumento de ventas del 20% o más en comparación con el año anterior**

In [3]:
aumento_ventas_20 = pd.read_sql_query(
    """
WITH ventas_anuales AS (
SELECT
store_id,
strftime('%Y', date) AS anio,
SUM(sales) AS total_ventas
FROM sales
GROUP BY 1,2
),
ventas_con_porcentaje AS (
SELECT
v1.store_id,
v1.anio AS anio_actual,
v1.total_ventas AS ventas_actuales,
v2.total_ventas AS ventas_previas,
ROUND(((v1.total_ventas - v2.total_ventas) / v2.total_ventas) * 100,2) AS porcentaje_cambio
FROM ventas_anuales v1
LEFT JOIN ventas_anuales v2
ON v1.store_id = v2.store_id 
AND v1.anio = CAST(v2.anio + 1 AS TEXT)
)
SELECT COUNT(DISTINCT store_id) AS tiendas_crecimiento_20
FROM ventas_con_porcentaje
WHERE porcentaje_cambio >= 20
    """, conn)
display(aumento_ventas_20)

,tiendas_crecimiento_20
0,30


## **Cuál fue el producto que experimentó la mayor variación de precio a lo largo del periodo de 2017 a 2019**

In [4]:
producto_mayor_variacion = pd.read_sql_query(
    """
SELECT product_id, 
min(price) precio_min, 
max(price) precio_max, 
max(price) - min(price) diferencia_precio
FROM sales s
WHERE price <> ""
GROUP BY 1
ORDER BY 4 DESC
LIMIT 1
    """, conn)
display(producto_mayor_variacion)

,product_id,precio_min,precio_max,diferencia_precio
0,P0632,549.9,849.9,300.0


## **En qué mes y año se registró la mayor cantidad de ingresos generados en todas las tiendas**

In [5]:
mayor_cantidad_ingresos_mes_ano = pd.read_sql_query(
    """
WITH AGRUPADO AS (
SELECT
store_id,
strftime('%Y', date) year,
strftime('%m', date) month,
sum(revenue) AS total_revenue
FROM sales
WHERE revenue <> ""
GROUP BY 1, 2, 3
),
RANKING AS (
SELECT 
store_id, 
YEAR, 
MONTH, 
total_revenue,
ROW_NUMBER() OVER ( PARTITION BY store_id ORDER BY total_revenue DESC) AS fila
FROM AGRUPADO
)
SELECT store_id, YEAR, MONTH, total_revenue
FROM RANKING
WHERE  fila= 1
ORDER BY store_id DESC
    """, conn)
display(mayor_cantidad_ingresos_mes_ano)

,store_id,YEAR,MONTH,total_revenue
0,S0144,2019,08,12977.97
1,S0143,2019,08,3322.20
2,S0142,2019,08,15162.83
3,S0141,2019,08,2191.69
4,S0140,2018,10,7716.60
...,...,...,...,...
139,S0005,2018,09,11160.12
140,S0004,2019,08,15797.19
141,S0003,2019,08,9584.09
142,S0002,2019,08,32383.31


## **Cuál fue el tamaño promedio de las tiendas en cada ciudad durante el año 2018**

In [8]:
promedio_tamaño_tiendas = pd.read_sql_query(
    """
WITH tiendas_unicas AS (
SELECT DISTINCT store_id 
FROM sales s
WHERE strftime('%Y', date) = "2018"
)
SELECT city_id, ROUND(AVG(store_size),2) store_size_avg
FROM store_cities s
LEFT JOIN tiendas_unicas t
ON s.store_id = t.store_id
GROUP BY 1
    """, conn)
pd.set_option('display.max_rows', 10)
display(promedio_tamaño_tiendas)

,city_id,store_size_avg
0,C001,20.0
1,C002,47.0
2,C003,13.0
3,C004,63.0
4,C005,19.0
...,...,...
32,C033,35.0
33,C034,16.0
34,C035,18.0
35,C036,43.0


![Evidencia](https://raw.githubusercontent.com/isantrich/workshop_DE/refs/heads/main/images_dbeaver/promedio_tama%C3%B1o_tiendas.png)

In [4]:
conn.close()